## Введение
Помимо базовых функций, в приложении также имеется премиум-подписка, которая дает доступ к ряду важных дополнительных возможностей. Был проведен A/B тест, в рамках которого для новых пользователей из нескольких стран была изменена стоимость премиум-подписки* при покупке через две новые платежные системы. При этом стоимость пробного периода оставалась прежней.

### Задача:

1. Проверить Был ли эксперимент успешен в целом.

2. Проанализируйте, имеет ли нововведение смысл среди каких-либо конкретных групп пользователей.

*Деньги за подписку списываются ежемесячно до тех пор, пока пользователь её не отменит.*


## Данные 

### users_*.csv – информация о пользователях

uid – идентификатор пользователя

age – возраст

attraction_coeff – коэффициент привлекательности (от 0 до 1000, лайки/просмотры *1000)

coins – число монеток (внутренняя валюта)

country – страна  

visit_days – в какие дни после регистрации пользователь посещал приложение (напр. в 1, затем в 7)

gender – пол

age_filter_start  – фильтр поиска, мин. значение 

age_filter_end  – фильтр поиска, макс. значение 

views_count – число полученных оценок 

was_premium – был ли когда-либо премиум (либо пробный период премиум-статуса, либо купленный за деньги)

is_premium –  является ли премиум

total_revenue – нормированная выручка


### transactions_*.csv – информация о платежах пользователей:

uid – идентификатор пользователя

country – страна

joined_at – дата и время регистрации

paid_at – дата и время покупки

revenue – нормированная выручка

payment_id – идентификатор платежа

from_page – откуда пользователь перешел на страницу оплаты

product_type – тип продукта (trial_premium – пробная премиум-подписка, premium_no_trial – премиум-подписка без пробной, coins – подписка за 
внутреннюю валюту, other_type – другое)


### Файлы:

users_test – информация о пользователях в тестовой группе. 

users_control_1 – информация о пользователях в первой контрольной группе. 

users_control_2 – информация о пользователях во второй контрольной группе . 

transactions_test – информация о платежах пользователей в тестовой группе. 

transactions_control_1 – информация о платежах пользователей в первой контрольной группе. 

transactions_control_2 – информация о платежах пользователей во второй контрольной группе 
  



In [54]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import datetime

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.stats import norm, mannwhitneyu

In [37]:
def processing_a_data(df):
    df.fillna('0', inplace=True)
    df['visit_days'] = df['visit_days'].apply (lambda x : [int(i) for i in x.replace('.',',').split(',') ] )
    df['count_of_visit_days'] = df['visit_days'].apply (lambda x : len(x) if x[0]!=0 else 0)
    lst_col = ['uid', 'age', 'attraction_coeff', 'coins', 'country', 'visit_days', 'count_of_visit_days',
       'gender', 'age_filter_start', 'age_filter_end', 'views_count',
       'was_premium', 'is_premium', 'total_revenue']
    df = df[lst_col]
    return df 



In [63]:
users_test = pd.read_csv('./users_test.csv',sep = ';') 
processing_a_data(users_test)
users_test.shape

(4308, 14)

In [39]:
users_control_1 = pd.read_csv('./users_control_1.csv', sep = ';')
processing_a_data(users_control_1)

,uid,age,attraction_coeff,coins,country,visit_days,count_of_visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892319115,25,435,0,United States of America,"[1, 2, 3, 4, 5, 6]",6,0,22,32,982,0,0,0
1,891248523,29,500,0,United States of America,"[1, 2]",2,1,26,32,12,0,0,0
2,891670932,33,800,0,France,"[1, 2, 3, 5, 6, 8, 9, 11, 13, 16]",10,1,30,36,5,0,0,0
3,891060786,26,0,0,Argentina,"[1, 2, 3, 4, 5]",5,1,23,29,0,0,0,0
4,892006554,35,294,0,United States of America,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 19]",14,1,30,40,17,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,892022304,26,227,0,United States of America,[0],0,0,23,29,427,0,0,0
4336,891905022,38,200,0,Argentina,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15...",16,1,20,50,5,0,0,0
4337,891304722,49,411,0,United States of America,[1],1,1,46,52,17,0,0,0
4338,891302328,27,0,0,United States of America,"[1, 3, 5]",3,1,24,30,0,0,0,0


In [40]:
users_control_2 = pd.read_csv('./users_control_2.csv', sep = ';')
processing_a_data(users_control_2)

,uid,age,attraction_coeff,coins,country,visit_days,count_of_visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892035504,37,137,0,Israel,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13]",11,1,30,40,51,0,0,0
1,891782112,57,0,0,Italy,[1],1,1,54,60,0,0,0,0
2,891110337,30,769,0,France,"[1, 2, 3, 4, 5]",5,1,27,33,13,0,0,0
3,891796320,48,750,0,France,"[1, 4, 6, 8, 10, 11, 12, 14, 15, 16, 18]",11,1,45,51,12,0,0,0
4,891880212,54,638,0,United States of America,[1],1,1,35,53,94,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4259,891937401,63,242,0,Italy,"[1, 2]",2,1,50,66,33,0,0,0
4260,891692844,35,500,0,France,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 14, 16]",13,1,24,45,2,0,0,0
4261,891096429,41,333,0,United States of America,[0],0,1,38,44,6,0,0,0
4262,891697584,48,210,0,United Kingdom (Great Britain),"[2, 3, 5, 7, 9, 10, 12, 14, 15]",9,1,45,51,19,0,0,0


In [82]:
transactions_test = pd.read_csv('./transactions_test.csv',sep=';')

transactions_test[['joined_at', 'paid_at']] = transactions_test[['joined_at', 'paid_at']].apply (pd.to_datetime)
transactions_test.head(3)

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891345942,Italy,2017-05-11 13:00:00,2017-11-13 15:04:00,12909,147,trial_vip_popup,trial_premium
1,892054251,United States of America,2017-10-22 00:33:00,2017-10-30 01:37:00,13923,147,trial_vip_popup,trial_premium
2,892236423,United States of America,2017-10-18 01:09:00,2017-10-23 00:15:00,3783,67,menu,other_type


In [102]:

transactions_control_1 = pd.read_csv('./transactions_control_1.csv', sep=';')
transactions_control_1[['joined_at', 'paid_at']] = transactions_control_1[['joined_at', 'paid_at']].apply (pd.to_datetime)
transactions_control_1.dropna(inplace=True)
transactions_control_1[['uid','revenue','payment_id']] = transactions_control_1[['uid','revenue','payment_id']].astype(int)
transactions_control_1.head(3)

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891319275,France,2017-05-11 19:57:00,2017-12-11 21:57:00,12727,147,trial_vip_popup,trial_premium
1,892421826,Israel,2017-10-14 12:46:00,2017-10-23 12:54:00,14586,147,trial_vip_popup,trial_premium
2,891822480,Italy,2017-10-26 22:27:00,2017-12-11 20:59:00,1911,19,none,coins


In [81]:
transactions_control_2 = pd.read_csv('./transactions_control_2.csv',sep=';')
transactions_control_2[['joined_at', 'paid_at']] = transactions_control_2[['joined_at', 'paid_at']].apply (pd.to_datetime)
transactions_control_2.head(3)

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891266616,Argentina,2017-06-11 15:25:00,2017-10-11 17:35:00,6305,19,autorefill,coins
1,892186737,Chile,2017-10-19 06:03:00,2017-04-11 08:53:00,4732,147,promo_09,premium_no_trial
2,891959004,United States of America,2017-10-24 00:12:00,2017-10-31 02:12:00,10153,147,trial_vip_popup,trial_premium
